<a href="https://colab.research.google.com/github/1kaiser/Positive/blob/main/%F0%9F%98%8ETestDough%E2%9C%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/1kaiser/Positive.git

In [ ]:
%cd /content/Positive

# Process to download

In [ ]:
!npm install -g npm
!npm install puppeteer turndown
!apt-get update
!apt install chromium-chromedriver

In [ ]:
%%writefile index.js

'use strict';

const puppeteer = require('puppeteer');
const fs = require('fs');
var TurndownService = require('turndown');

(async() => {
  const browser = await puppeteer.launch({executablePath:"/usr/lib/chromium-browser/chromium-browser", args:['--no-sandbox']});
  const page = await browser.newPage();

    await page.goto("https://www.whatdoesitmean.com/index632.htm", {
        waitUntil: "networkidle0",
    })

    const links = await page.$eval("body > div.section1", (el) => el.childElementCount);
    console.log(links)
    let golinks = [];
    let gg
    for (let link = 12; link <= links; link++) {
        if (link == 80 | link == 445 | link == 444 | link == 812 | link == 1165 | link == 1349 | link == 1468 | link == 1470 | link == 1742 | link == 1744 | link == 1970 | link == 1972 | link == 1975) {
            link++
        } else {
            gg = ''
            gg = await page.evaluate((link) => {
                return document.querySelector("body > div > p:nth-child(" + link + ") > b").getAttribute("style");
            }, link)
            if (gg != '' && gg == "mso-bidi-font-weight:normal") {
                //console.log(gg, link)
                let title = ''
                title = await page.evaluate((link) => {
                    return document.querySelector("body > div > p:nth-child(" + link + ")").innerText
                }, link)
                let gotolink = ''
                gotolink = await page.evaluate((link) => {
                    return document.querySelector("body > div > p:nth-child(" + link + ") > b > span > a").getAttribute('href')
                }, link)
                if (title != null && gotolink != null) {
                    //    console.log(title, gotolink)

                    golinks.push({
                        Title: title.replace(/[`~!@#$%^&*()_|+\-=?;:'"’”“,.<>\{\}\[\]\\\/]/gi, '_'),
                        Link: gotolink
                    })

                }
            }
        }



    }
    // console.log(golinks[0]["Link"], golinks[0]["Title"])
    let file = '\out.txt'
    if (!fs.existsSync(file)) {
        fs.writeFileSync(file, '', function (err) {
            if (err)
                console.log(err);
        });
    }
    var darray = fs.readFileSync('\out.txt', 'utf8').split('\n');
    let file1 = "MD"
    if (!fs.existsSync(file1)) {
        fs.mkdirSync(file1, function (err) {
            if (err)
                console.log(err);
        });
    }
    for (let i = 0; i <= golinks.length; i++) {
        // console.log(golinks[i]["Link"],'\n',golinks[i]["Title"])
        let completed = []
        if (!darray.join().includes(golinks[i]["Link"])) {

            let pagedetails = ''
            var turndownService = new TurndownService()
            const collectData = async (page) => {
                try {
                    await page.goto(golinks[i]["Link"], {
                        waitUntil: "networkidle0"
                    });
                    return await page.content();
                } catch (err) {
                    console.error(err.message);
                    return false;
                }
            }

            let data = false;
            let attempts = 0;
            while (data == false && attempts < 2) {
                data = await collectData(page);
                attempts += 1;
                if (data != false) {
                    pagedetails = data
                    pagedetails = turndownService.turndown(pagedetails)
                    fs.writeFile(`MD/${golinks[i]["Title"]}.md`, pagedetails, err => {
                        if (err) console.log(err)
                    })

                    completed.push(golinks[i]["Link"].concat('\n'))
                    fs.appendFileSync('out.txt', completed.toString())
                } else {
                    i++
                }
            }
        }
        console.log("going to next link " + i + "")

    }


})()

In [ ]:
!node index.js

#zipping

In [ ]:
!zip -r /content/20220313.zip /content/Positive/MD /content/Positive/out.txt